In [ ]:
from selenium import webdriver
import time 
import random
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import os
import requests

In [ ]:
os.getcwd()

In [ ]:
driver = webdriver.Chrome("C:/Users/i7-870/chromedriver.exe")
#driver = webdriver.Chrome("C:/Users/User/Documents/Julie/chromedriver.exe")

In [ ]:
url = "https://www.wsj.com"
driver.get(url)

In [ ]:
#sign in
e = driver.find_element_by_css_selector('#root > div > div > div > div.WSJTheme--margin-bottom--2-lor3Ur.styles--margin-bottom--1qLtxtgQ > header > div.style--login-buttons--3iPE-lSo.style--clearfix--P-MgmEGt > a:nth-child(2)')
e.click()

In [ ]:
#輸入email
q = driver.find_element_by_name('username')
q.send_keys('kenneth.wy.lee21@gmail.com')

#輸入密碼
q = driver.find_element_by_name('password')
q.send_keys('qwer0987')

#sign in
e = driver.find_element_by_css_selector('#basic-login > div:nth-child(1) > form > div > div:nth-child(6) > div.sign-in.hide-if-one-time-linking > button')
e.click()

In [ ]:
#Abbreviate company name
sp_500 = pd.read_csv('WSJ S&P 500 成分股.csv')

#["Co.", "Corp.", "Cos."]
keyword = []
for i in sp_500["Company"]:
    company = i.replace(" Cl A", "").replace(" Cl B", "").replace(" Cl C", "")\
                .replace(" Series A","").replace(" Series C","")\
                .replace(" Inc.", "")\
                .replace(" PLC", "")\
                .replace(" Ltd.", "")\
                .replace(".com", "")\
                .replace(" N.A.", "")\
                .replace(" N.V.", "")
    keyword.append(company)
sp_500["Company"] = keyword
sp_500

In [ ]:
#初始化query，搜尋時使用
sp_500["query1"] = sp_500["Company"].apply(lambda x: x.replace(" ", "%20")\
                                                   .replace("'", "%27")\
                                                   .replace("&", "%26"))
sp_500["query1"] = sp_500["query1"].apply(lambda x: "%22" + x + "%22")

sp_500["query2"] = sp_500["Symbol"].apply(lambda x: "%22" + x + "%22")
sp_500

# Links

In [ ]:
for i in range(3):#len(sp_500)):
    
    #建立新的資料夾，名稱是company
    dirpath = "C:\\Users\\i7-870\\Documents\\Python\\news\\WSJ\\news\\"
    path = os.path.join(dirpath, sp_500["Company"][i])

    if not os.path.exists(path):#如果沒有該路徑
        os.makedirs(path)#就建立該路徑
    #print("創建資料夾:", path)
    
    #連接url
    url0 = "https://www.wsj.com/search?query={query}&isToggleOn=true&operator=AND&sort=date-desc&duration=4y&startDate=2005%2F05%2F05&endDate=2021%2F06%2F08&source=%2Cwsjpro%2Cwsjie"\
    .format(query = sp_500["query1"][i])
    
    driver.get(url0)
    
    time.sleep(random.uniform(3,6))
    
    
    '''抓出搜尋頁數'''
    #查看該公司的搜尋結果有幾頁
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        try:
            #抓頁數
            pages = soup.find("div", "WSJTheme--SearchResultPagination--2_RDsqbb").find_all("span")[-1].text
            pages = pages.split(' ')[1]
            print(i, sp_500["Company"][i], " Pages: ", pages)
        except:
            if soup.find("title").text == "Page Not Found":
                time.sleep(random.uniform(150, 200))
            
            #有的時候driver.get()會連不上，因此放外面，方便同時給if或連不上時使用
            driver.get(url0)
            time.sleep(random.uniform(3,6))  
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            #抓頁數
            pages = soup.find("div", "WSJTheme--SearchResultPagination--2_RDsqbb").find_all("span")[-1].text
            pages = pages.split(' ')[1]
            print(i, sp_500["Company"][i], " Pages: ", pages)
    except:
        #no result，跳下一家公司
        if soup.find("div", "WSJTheme--no-result--mNcujucR").text == "No articles or videos have been found.":
            print(i, sp_500["Company"][i], " no result")
            continue
    
    
    '''抓出文章'''
    #依照頁數逐頁抓取
    df = []
    for page in range(1, int(pages)+1):
        
        #連接加了頁數的url
        url = url0 + "&page=" + str(page)
        driver.get(url)

        time.sleep(random.uniform(4,6))
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        #如果抓不到article，就等幾分鐘再重新連接並返回url檢查
        while True:
            ''''''
            elements = soup.find_all("article", "WSJTheme--story--XB4V2mLz WSJTheme--overflow-visible--3OB31tWq WSJTheme--border-bottom--s4hYCt0s")
            ''''''
            if elements == []: #如果沒有news         
                print(url)
                time.sleep(random.uniform(150, 200))
                driver.get(url)#重新連接
                time.sleep(random.uniform(4,6))
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                continue
            else:#如果有news
                break
        
        
        for ele in elements:
            headline = ele.find("span", "WSJTheme--headlineText--He1ANr9C").text
            link = ele.find("a")['href']
            time_ = ele.find("div", "WSJTheme--timestamp--2zjbypGD").text
            
            if time_ == None:
                break
                
            try:
                category = ele.find("li", "WSJTheme--type--3JhCic1c").text
            except:
                category = ""
            
            df.append({
                "Headline": headline,
                "Links": link,
                "Time": time_,
                "Category": category,
            })
        if time_ == None:
            break
    if time_ == None:
        break       
    #save
    pd.DataFrame(df).to_csv(os.path.join(path, sp_500["Company"][i]) + "_company_url.csv",index = False)

# Content

In [ ]:
datapath = []
path = os.path.join(os.getcwd(), 'news')
print(path)
for dirname in os.listdir(path):#資料夾裡的子資料夾
    dir_path = os.listdir(os.path.join(path, dirname))
     
    for filename in dir_path:
        if "_company_url.csv" in filename:
            datapath.append(os.path.join(path, dirname, filename))
print(len(datapath))
datapath

In [ ]:
for i in range(0, len(datapath)):
    print("-----------------------------------------------------------------------------------")
    print(i, datapath[i])
    data = pd.read_csv(datapath[i])#讀取公司相關新聞的url
    
    df = []
    erro=[]
    for url in range(len(data["Links"])):#依序抓出url
        driver.get(data["Links"][url])
        time.sleep(2) 
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')#讀取網站的html
        
        
        #確認是WSJ Articles還是WSJ PRO Articles
        source = "articles"
        if soup.find("a", "style--logo-1t5fLHT-VmzJTtPNZR0C0P"):
            source = "pro articles"
            
            
        '''確認article是不是我們要的類型'''
        #如果是出售報價不是news，就跳過
        if soup.find("hr", "web-ui-slideshow-hr"):
            print("----", url, "Sales", data["Links"][url])
            continue

        #如果是Noted，就跳過
        if soup.find("a", "style--section-logo-1OPM_iWTXp3KPZjnR-sGBU style--noted-3D4iJQdRyI-61o3KCcIwAB style--section-logo-37slLHd9h2vhlXUDQBDu6n"):
            print("----", url, "Noted", data["Links"][url])
            continue
        
        '''確定有進入文章'''
        #連接失敗才執行
        count = 0
        while True:#連線太多次會失敗
            try :#連接成功
                try:
                    h1 = soup.find("h1", "wsj-article-headline").text
                    break
                except:
                    try:
                        h1 = soup.find("h1", "bigTop__hed").text
                        break
                    except:
                        h1 = soup.find("h1", "headline").text
                        break
            except:#如果連接失敗
                count += 1
                if count > 5:#如過持續重新載入失敗，代表網頁無回應
                    erro.append([datapath[i], url, "unconnect", data["Links"][url]])
                    break
                print(url, "unconnect", data["Links"][url])
                time.sleep(200)#等幾分鐘
                driver.get(data["Links"][url])#重新連接
                time.sleep(2)
                soup = BeautifulSoup(driver.page_source, 'html.parser')#讀取網站的html
                continue
        
        if count > 5:#如過持續重新載入失敗，代表網頁無回應
            break
            
            
        '''text'''
        text = str()
        try:
            try:
                element = soup.find("div", "article-content").find_all("p")
                for ele in element:
                    text += ele.text.replace("\n", '').replace("  ", '')
            except:
                try:
                    element = soup.find("div", "nc-exp-article").find_all("p")
                    for ele in element:
                        text += ele.text.replace("\n", '').replace("  ", '')
                except:
                    try:
                        element = soup.find_all("h3", "section-title")
                        for ele in element:
                            text += ele.text.replace("\n", '').replace("  ", '')

                        if text == "":
                            raise
                    except:
                        #pro article
                        element = soup.find("div", "wsj-snippet-body").find_all("p")
                        for ele in element:
                            text += ele.text.replace("\n", '').replace("  ", '')
        except:
            print(url, "text", data["Links"][url])
            erro.append([datapath[i], url, "text", data["Links"][url]])
            continue
        
        
        #將資料存入DataFrame
        df.append({
            "headline": data["Headline"][url],
            "time": data["Time"][url],
            "link": data["Links"][url],
            "text": text,
            "category": data["Category"][url],
            "source": source,
        })
        
        
    #save corpus
    path = datapath[i].replace("_url", "")#刪除url的部分
    pd.DataFrame(df).to_csv(path, index = False)#save
    
    
    #save erro
    erro0 = pd.read_csv("erro.csv")
    erro = pd.DataFrame(erro, columns=["file", "index",'problem', 'link'])
    erro = erro0.append(erro, ignore_index = True)
    erro.to_csv('erro.csv',index = False)#save